# Demo Code: Export&Import DataStage flow

## Prerequisite
Create your own yaml file based on [configuration_template.yaml](./configuration_template.yaml), and modify "CONFIG_FILE" below


In [2]:
import sys
import os
import yaml
import subprocess

In [3]:
# point to a local file with credentials. It is not synced to git.
CONFIG_FILE = "export_import_value_set.yaml"

In [4]:
# Load parameters from the YAML file
with open(CONFIG_FILE, 'r') as file:
    config = yaml.safe_load(file)

# print(config)
DSJOB_URL = config['url']
DSJOB_USER = config['user']
DSJOB_PWD = config['password']
EXPORT_PRJ_NAME = config['export_prj_name']
if "export_zip_name" in config:
    EXPORT_ZIP_NAME = config['export_zip_name']
else:
    EXPORT_ZIP_NAME = EXPORT_PRJ_NAME+"_valueset.zip"

parameter_sets = config['parameter_sets']

IMPORT_PRJ_NAME = config['import_prj_name']
print("EXPORT_PRJ_NAME=",EXPORT_PRJ_NAME)
print("IMPORT_PRJ_NAME=",IMPORT_PRJ_NAME)

EXPORT_PRJ_NAME= Multicloud Data Integration L3 Tech Lab
IMPORT_PRJ_NAME= DataStage Import


In [5]:
# Enable dsjob
%env CPDCTL_ENABLE_DSJOB=true
%env CPDCTL_ENABLE_DATASTAGE=true
%env CPDCTL_ENABLE_VOLUMES=1

env: CPDCTL_ENABLE_DSJOB=true
env: CPDCTL_ENABLE_DATASTAGE=true
env: CPDCTL_ENABLE_VOLUMES=1


In [6]:
# Configure cpdctl with the parameters
!cpdctl config user set CP4D-user --username "$DSJOB_USER" --password "$DSJOB_PWD"
!cpdctl config profile set CP4D-profile --url "$DSJOB_URL" --user CP4D-user
!cpdctl config profile use CP4D-profile

Switched to profile "CP4D-profile".


In [10]:
!cpdctl dsjob list-flows --project "$EXPORT_PRJ_NAME" 

...
Employee Ranking

Total: 1 Flows

Status code = 0


### **get-flow**  will export specified flow 

In [8]:
# Only one parameter set and value set are supported
!cpdctl dsjob get-flow --project "$EXPORT_PRJ_NAME" --name "Employee Ranking"  --output file --file-name "Employee Ranking.json"
!cat "Employee Ranking.json"

...

Status code = 0
{
    "app_data": {
        "additionalProperties": {
            "ELTDropdown": {
                "id": "nesting",
                "label": "Generate nested SQL",
                "value": "NESTED_QUERY"
            },
            "enableRCP": false,
            "enableSchemaLessDesign": false,
            "globalAcp": true,
            "isDefineParameterToggled": false,
            "rcpLinkList": [
                "af450aa6-6c83-44fa-87d3-9fc3af7e0efd"
            ],
            "runMode": "ETL",
            "transInputLinkMapper": {}
        },
        "datastage": {
            "date_format": "",
            "decimal_separator": "",
            "flowRunPriorityQueue": "",
            "message_handlers": [],
            "subroutines": {
                "function_calls": [
                    {
                        "call_context": "BEFORE",
                        "function_name": "ExecSH",
                        "input_value": "#mkdir -p /px-storage/pip_modul

#### We don't have **update-flow**, hence we need to delete-flow then create-flow

In [12]:
# [optional] delete valueset
!cpdctl dsjob delete-flow --project "$IMPORT_PRJ_NAME" --name "Employee Ranking"

...
ValueSet Deleted from Paramset ID:  eff877e6-bc4f-4da9-8f58-6f16faadede3

Status code = 0


In [11]:
# If ValueSet doesn't exist, we create
!cpdctl dsjob create-flow --project "$IMPORT_PRJ_NAME" --name "Employee Ranking" --pipeline-file "Employee Ranking.json"

...
2472e274-6757-4cc7-ae3e-1351daf93bef

Status code = 0
